**Henrique Gomes de Sousa <br>**
**Magno Araujo de Sousa<br>**
**Sthefani Simões dos Santos<br>**

In [0]:
# Biblioteca necessarias para o algoritmo
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [0]:
#Importando a base de dados
# DataFrame utilizado para cálculo do modelo de hipertensão
df_base_pes = pd.read_csv('ARQUIVO_SAIDA_VIGITEL_vr5.csv',delimiter=';',encoding='UTF-8') 

In [3]:
#Drop da coluna IN_ALVO_DIABETES
df_base_pes.drop('IN_ALVO_DIABETES', axis=1, inplace=False)
#informações sobre a base de dados
df_base_pes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52395 entries, 0 to 52394
Columns: 190 entries, GR_IDADE_18_26_ANOS to IN_ALVO_DIABETES
dtypes: float64(4), int64(186)
memory usage: 76.0 MB


In [0]:
#DataFrame que receberá os resultados dos modelos pra comparação
df_comp = pd.DataFrame(columns=['*******************', 'HIPERTENSÃO', 'DIABETES'])

# **1) PRÉ-PROCESSAMENTO DOS DADOS**

In [0]:
#Retirando as linhas com valores nulos da base
df_base_pes.dropna(axis=0,how='any',inplace = True)

In [6]:
#informações sobre a base de dados
df_base_pes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47653 entries, 0 to 52394
Columns: 190 entries, GR_IDADE_18_26_ANOS to IN_ALVO_DIABETES
dtypes: float64(4), int64(186)
memory usage: 69.4 MB


# **PRE-PROCESSAMENTO DA BASE PARA HIPERTENSAO**

In [0]:
# Dividindo os dados em treino e teste:
X_train, X_test, y_train, y_test = train_test_split(df_base_pes, df_base_pes['IN_ALVO_HIPERTENSAO'],test_size =0.2)

In [8]:
# Verificando as formas dos dados dos conjuntos:
print('Conjunto de Treinamento: {} {}'.format( X_train.shape, y_train.shape))
print('Conjunto de Teste:       {} {}'.format(X_test.shape,y_test.shape))

Conjunto de Treinamento: (38122, 190) (38122,)
Conjunto de Teste:       (9531, 190) (9531,)


**1.1) BALANCEANDO CONJUNTO DE TREINAMENTO COM UNDERSAMPLING E OVERSAMPLING**

In [9]:
# DATASET DESBALANCEADO - Contagem das classes
qtde_classe_0, qtde_classe_1 = X_train.IN_ALVO_HIPERTENSAO.value_counts()
print('DATASET DESBALANCEADO\n  Total Classe 0 (NÃO): {}\n  Total Classe 1 (SIM): {}\n'.format(qtde_classe_0, qtde_classe_1))

DATASET DESBALANCEADO
  Total Classe 0 (NÃO): 25224
  Total Classe 1 (SIM): 12898



In [0]:
# Dividindo o dataset original por classes em dois DataFrames
df_class_0 = X_train[X_train['IN_ALVO_HIPERTENSAO'] == 0]
df_class_1 = X_train[X_train['IN_ALVO_HIPERTENSAO'] == 1]

**1.1.1.) OVERSAMPLING DA CLASSE 1 - AUMENTO DA CLASSE 1 EM 50% DA DIFERENÇA ATÉ A QTDE DA CLASSE 0**

In [0]:
import numpy as np
# Cria uma lista com os índices do DataFrame das Classes 1
list_index = list(df_class_1.index)

In [0]:
# armazena numa nova lista 50% dos indíces da list_index  
list_indexes = np.random.choice(list_index, int((len(df_class_0) - len(df_class_1))/2))

In [0]:
# concatena ao DataFrame df_class_1 os registros aleatórios desse mesmo DataFrame contidos em list_indexes
df_class_1 = pd.concat([df_class_1, df_class_1.loc[list_indexes]], axis=0)

In [0]:
# Atualiza o Conjunto de treinamento concatenando o 
X_train = pd.concat([df_class_0, df_class_1], axis=0)

In [15]:
# DATASET PARCIALMENTE BALANCEADO - Contagem das classes
qtde_classe_0, qtde_classe_1 = X_train.IN_ALVO_HIPERTENSAO.value_counts()
print('DATASET DESBALANCEADO\n  Total Classe 0 (NÃO): {}\n  Total Classe 1 (SIM): {}\n'.format(qtde_classe_0, qtde_classe_1))

DATASET DESBALANCEADO
  Total Classe 0 (NÃO): 25224
  Total Classe 1 (SIM): 19061



**1.1.2.) UNDERSAMPLING DA CLASSE 0 - REDUÇÃO DA CLASSE 0 ATÉ ATINGIR A QTDE DA CLASSE 1**

In [0]:
#Balanceando com undersampling e criando e recriando o DataFrame df_base_pes
df_class_0_under = df_class_0.sample(qtde_classe_1)
X_train = pd.concat([df_class_0_under, df_class_1], axis=0)

In [17]:
# DATASET BALANCEADO - Contagem das classes
qtde_classe_0, qtde_classe_1 = X_train.IN_ALVO_HIPERTENSAO.value_counts()
print('DATASET BALANCEADO\n  Total Classe 0 (NÃO): {}\n  Total Classe 1 (SIM): {}\n'.format(qtde_classe_0, qtde_classe_1))

DATASET BALANCEADO
  Total Classe 0 (NÃO): 19061
  Total Classe 1 (SIM): 19061



**1.1.3) AJUSTES FINAIS NOS DATAFRAMES DE TREINO E TESTE**
* y_train e y_test: Manutenção apenas da variável alvo
* X_train e X_test: Remoção da variável ALVO e manutenção das demais

In [0]:
y_train = X_train['IN_ALVO_HIPERTENSAO']
y_test = X_test['IN_ALVO_HIPERTENSAO']
X_train = X_train.drop('IN_ALVO_HIPERTENSAO',axis=1)
X_test = X_test.drop('IN_ALVO_HIPERTENSAO',axis=1)

In [19]:
# Verificando novamente as formas dos dados dos conjuntos:
print('Conjunto de Treinamento (HIPER): {} {}'.format( X_train.shape, y_train.shape))
print('Conjunto de Teste (HIPER):       {} {}'.format(X_test.shape,y_test.shape))

Conjunto de Treinamento (HIPER): (38122, 189) (38122,)
Conjunto de Teste (HIPER):       (9531, 189) (9531,)


In [20]:
df_base_pes.head()

,GR_IDADE_18_26_ANOS,GR_IDADE_27_35_ANOS,GR_IDADE_36_44_ANOS,GR_IDADE_45_53_ANOS,GR_IDADE_54_62_ANOS,GR_IDADE_63_71_ANOS,GR_IDADE_MAIS_71_ANOS,TP_SEXO_MASC,TP_SEXO_FEM,GR_ANOS_EST_ATE_9,GR_ANOS_EST_10_14,GR_ANOS_EST_MAIOR_15,IN_EXC_PESO,IN_OBESIDADE_VIG,IN_FREQ_HORT_1,IN_FREQ_HORT_2,IN_FREQ_HORT_3,IN_FREQ_HORT_4,IN_FREQ_HORT_5,IN_FREQ_HORT_6,IN_FREQ_HORT_7,IN_FREQ_HORT_8,IN_FREQ_HORT_9,IN_FREQ_HORT_10,IN_FREQ_HORT_11,IN_FREQ_SUCO_FRUTA_1,IN_FREQ_SUCO_FRUTA_2,IN_FREQ_SUCO_FRUTA_3,IN_FREQ_SUCO_FRUTA_4,IN_FREQ_SUCO_FRUTA_5,IN_FREQ_SUCO_FRUTA_6,IN_FREQ_SUCO_FRUTA_7,IN_FREQ_SUCO_FRUTA_8,IN_FREQ_SUCO_FRUTA_9,IN_FREQ_SUCO_FRUTA_10,IN_FREQ_SUCO_FRUTA_11,IN_FREQ_SUCO_FRUTA_12,IN_FREQ_SUCO_FRUTA_13,IN_FREQ_SUCO_FRUTA_14,IN_FREQ_SUCO_FRUTA_15,...,IN_EXER_FIS_1,IN_EXER_FIS_2,IN_EXER_FIS_3,IN_EXER_FIS_4,IN_EXER_FIS_5,IN_EXER_FIS_6,IN_EXER_FIS_7,IN_EXER_FIS_8,IN_EXER_FIS_9,IN_EXER_FIS_10,IN_EXER_FIS_11,IN_EXER_FIS_12,IN_EXER_FIS_13,IN_EXER_FIS_14,IN_EXER_FIS_15,IN_EXER_FIS_16,IN_EXER_FIS_17,IN_EXER_FIS_18,IN_EXER_FIS_19,IN_EXER_FIS_20,IN_EXER_FIS_21,IN_EXER_FIS_22,IN_EXER_FIS_23,IN_EXER_FIS_24,IN_EXER_FIS_25,IN_EXER_FIS_26,IN_EXER_FIS_27,IN_EXER_FIS_28,IN_EXER_FIS_29,IN_EXER_FIS_30,IN_FREQ_FUMANTE_1,IN_FREQ_FUMANTE_2,IN_FREQ_FUMANTE_3,IN_FREQ_FUMANTE_4,IN_FREQ_FUMANTE_5,IN_FREQ_FUMANTE_6,IN_FREQ_FUMANTE_7,IN_FREQ_FUMANTE_8,IN_ALVO_HIPERTENSAO,IN_ALVO_DIABETES
0,1,0,0,0,0,0,0,0,1,0,0,1,2.0,2.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0
1,0,0,0,0,0,1,0,1,0,0,0,1,1.0,2.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0
2,0,0,0,1,0,0,0,0,1,0,0,1,1.0,2.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
3,0,0,0,0,1,0,0,0,1,0,0,1,1.0,2.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,0.0
4,0,0,0,0,1,0,0,0,1,0,0,1,1.0,2.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1.0,0.0


# **PRE-PROCESSAMENTO PARA BASE DE DIABETICOS**

In [0]:
# Dividindo os dados em treino e teste:
X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes = train_test_split(df_base_pes, df_base_pes['IN_ALVO_DIABETES'],test_size =0.2)

In [22]:
# Verificando as formas dos dados dos conjuntos:
print('Conjunto de Treinamento: {} {}'.format( X_train_diabetes.shape, y_train_diabetes.shape))
print('Conjunto de Teste:       {} {}'.format(X_test_diabetes.shape,y_test_diabetes.shape))

Conjunto de Treinamento: (38122, 190) (38122,)
Conjunto de Teste:       (9531, 190) (9531,)


**1.1) BALANCEANDO CONJUNTO DE TREINAMENTO COM UNDERSAMPLING E OVERSAMPLING**

In [23]:
# DATASET DESBALANCEADO - Contagem das classes
qtde_classe_0, qtde_classe_1 = X_train_diabetes.IN_ALVO_DIABETES.value_counts()
print('DATASET DESBALANCEADO\n  Total Classe 0 (NÃO): {}\n  Total Classe 1 (SIM): {}\n'.format(qtde_classe_0, qtde_classe_1))

DATASET DESBALANCEADO
  Total Classe 0 (NÃO): 34035
  Total Classe 1 (SIM): 4087



In [0]:
# Dividindo o dataset original por classes em dois DataFrames
df_diab_class_0 = X_train_diabetes[X_train_diabetes['IN_ALVO_DIABETES'] == 0]
df_diab_class_1 = X_train_diabetes[X_train_diabetes['IN_ALVO_DIABETES'] == 1]

In [0]:
import numpy as np
# Cria uma lista com os índices do DataFrame das Classes 1
list_index = list(df_diab_class_1.index)

In [0]:
# armazena numa nova lista 50% dos indíces da list_index  
list_indexes = np.random.choice(list_index, int((len(df_diab_class_0) - len(df_diab_class_1))/2))

In [0]:
# concatena ao DataFrame df_class_1 os registros aleatórios desse mesmo DataFrame contidos em list_indexes
df_diab_class_1 = pd.concat([df_diab_class_1, df_diab_class_1.loc[list_indexes]], axis=0)

In [0]:
# Atualiza o Conjunto de treinamento concatenando o 
X_train_diabetes = pd.concat([df_diab_class_0, df_diab_class_1], axis=0)

In [29]:
# DATASET PARCIALMENTE BALANCEADO - Contagem das classes
qtde_classe_0, qtde_classe_1 = X_train_diabetes.IN_ALVO_DIABETES.value_counts()
print('DATASET DESBALANCEADO\n  Total Classe 0 (NÃO): {}\n  Total Classe 1 (SIM): {}\n'.format(qtde_classe_0, qtde_classe_1))

DATASET DESBALANCEADO
  Total Classe 0 (NÃO): 34035
  Total Classe 1 (SIM): 19061



**1.1.2.) UNDERSAMPLING DA CLASSE 0 - REDUÇÃO DA CLASSE 0 ATÉ ATINGIR A QTDE DA CLASSE 1**

In [0]:
#Balanceando com undersampling e criando e recriando o DataFrame df_base_pes
df_diab_class_0_under = df_diab_class_0.sample(qtde_classe_1)
X_train_diabetes = pd.concat([df_diab_class_0_under, df_diab_class_1], axis=0)

In [31]:
# DATASET BALANCEADO - Contagem das classes
qtde_classe_0, qtde_classe_1 = X_train_diabetes.IN_ALVO_DIABETES.value_counts()
print('DATASET BALANCEADO\n  Total Classe 0 (NÃO): {}\n  Total Classe 1 (SIM): {}\n'.format(qtde_classe_0, qtde_classe_1))

DATASET BALANCEADO
  Total Classe 0 (NÃO): 19061
  Total Classe 1 (SIM): 19061



**1.1.3) AJUSTES FINAIS NOS DATAFRAMES DE TREINO E TESTE**
* y_train e y_test: Manutenção apenas da variável alvo
* X_train e X_test: Remoção da variável ALVO e manutenção das demais

In [0]:
y_train_diabetes = X_train_diabetes['IN_ALVO_DIABETES']
y_test_diabetes = X_test_diabetes['IN_ALVO_DIABETES']
X_train_diabetes = X_train_diabetes.drop('IN_ALVO_DIABETES',axis=1)
X_test_diabetes = X_test_diabetes.drop('IN_ALVO_DIABETES',axis=1)

In [33]:
# Verificando novamente as formas dos dados dos conjuntos:
print('Conjunto de Treinamento (DIAB): {} {}'.format( X_train_diabetes.shape, y_train_diabetes.shape))
print('Conjunto de Teste (DIAB):       {} {}'.format(X_test_diabetes.shape,y_test_diabetes.shape))

Conjunto de Treinamento (DIAB): (38122, 189) (38122,)
Conjunto de Teste (DIAB):       (9531, 189) (9531,)


# **2) CRIAÇÃO DO MODELO DE ÁRVORE DE DECISÃO**

In [0]:
#from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Criação do DataFrame Vazio para armazenamento de todas as configurações e resultados dos modelos de 
#árvore de decisão implementados
df_scores = pd.DataFrame(columns=['x_depht', 'y_split', 'accuracy_score', 'MC - NAO(N/S)', 'MC - SIM (S/N)', 'Matriz Conf NAO', 'Matriz Conf SIM', 'Criterion', 'ALVO'])

**2.1) FUNÇAO DE INSTANCIAÇÃO DE CLASSIFICADORES**

In [0]:
# Função de instanciação de classificadores
def func_instancia_classif(TP_CRITERION, NU_DEPHT_INI, NU_DEPHT_FIM, NU_DEPHT_INC, NU_SPLIT_INI, NU_SPLIT_FIM, NU_SPLIT_INC, 
                           Y_TRAIN, Y_TEST, X_TRAIN, X_TEST, IN_ALVO):
    print('\nRESULTADOS COM {}\n'.format(TP_CRITERION.upper()))

    # Instânciando objeto classificador:
    for x_depht in range(NU_DEPHT_INI, NU_DEPHT_FIM, NU_DEPHT_INC):
        for y_split in range(NU_SPLIT_INI, NU_SPLIT_FIM, NU_SPLIT_INC):
            clf = DecisionTreeClassifier(criterion = TP_CRITERION, max_depth = x_depht, min_samples_split = y_split)
            clf = clf.fit(X_TRAIN, Y_TRAIN)
            resultado = clf.predict(X_TEST)
            #Calcula a acurácia
            acuracia = accuracy_score(Y_TEST, resultado)
            #Retorna a Matriz de Confusão
            matriz_confusao = confusion_matrix(Y_TEST, resultado)
            erro_matriz_nao = 100 - (100 * (matriz_confusao[0][1] / (matriz_confusao[0][0] + matriz_confusao[0][1])))
            erro_matriz_sim = 100 - (100 * (matriz_confusao[1][0] / (matriz_confusao[1][0] + matriz_confusao[1][1])))

            #Insere o registro no DataFrame
            df_scores.loc[len(df_scores)] = [x_depht, y_split, acuracia, erro_matriz_nao, erro_matriz_sim,  matriz_confusao[0], matriz_confusao[1], TP_CRITERION, IN_ALVO]
            print('Depht: {} | Split: {} | Acurácia: {} | ERRO MC-NAO(N/S): {} | ERRO MC-SIM(S/N): {} | '
                'MC-NAO(N/S): {} | MC-SIM (S/N): {} | Criterion: {}, | ALVO: {}'
                .format(x_depht, y_split, acuracia, erro_matriz_nao, erro_matriz_sim, matriz_confusao[0], matriz_confusao[1], TP_CRITERION, IN_ALVO))

**2.2) CHAMADA DA FUNÇÃO DE INSTANCIAÇÃO DE CLASSIFICADORES**

In [0]:
# Configurações de profundidade da árvore
NU_DEPHT_INI = 5
NU_DEPHT_FIM = 51
NU_DEPHT_INC = 5

# Configuração corte da árvore
NU_SPLIT_INI = 200
NU_SPLIT_FIM = 2001
NU_SPLIT_INC = 100


**2.2.1) Criação de Modelos com Árvore de Decisão (HIPERTENSÃO)**

In [37]:
# Criação de modelos com ENTROPY
TP_CRITERION = 'entropy'
func_instancia_classif(TP_CRITERION, NU_DEPHT_INI, NU_DEPHT_FIM, NU_DEPHT_INC, NU_SPLIT_INI, NU_SPLIT_FIM, NU_SPLIT_INC,
                       y_train, y_test, X_train, X_test, 'HIPERTENSAO')


RESULTADOS COM ENTROPY

Depht: 5 | Split: 200 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.61904761904762 | ERRO MC-SIM(S/N): 79.10863509749304 | MC-NAO(N/S): [4008 2292] | MC-SIM (S/N): [ 675 2556] | Criterion: entropy, | ALVO: HIPERTENSAO
Depht: 5 | Split: 300 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.61904761904762 | ERRO MC-SIM(S/N): 79.10863509749304 | MC-NAO(N/S): [4008 2292] | MC-SIM (S/N): [ 675 2556] | Criterion: entropy, | ALVO: HIPERTENSAO
Depht: 5 | Split: 400 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.65079365079365 | ERRO MC-SIM(S/N): 79.04673475704116 | MC-NAO(N/S): [4010 2290] | MC-SIM (S/N): [ 677 2554] | Criterion: entropy, | ALVO: HIPERTENSAO
Depht: 5 | Split: 500 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.65079365079365 | ERRO MC-SIM(S/N): 79.04673475704116 | MC-NAO(N/S): [4010 2290] | MC-SIM (S/N): [ 677 2554] | Criterion: entropy, | ALVO: HIPERTENSAO
Depht: 5 | Split: 600 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S)

In [38]:
# Criação de modelos com GINI
TP_CRITERION = 'gini'
func_instancia_classif(TP_CRITERION, NU_DEPHT_INI, NU_DEPHT_FIM, NU_DEPHT_INC, NU_SPLIT_INI, NU_SPLIT_FIM, NU_SPLIT_INC,
                       y_train, y_test, X_train, X_test, 'HIPERTENSAO')


RESULTADOS COM GINI

Depht: 5 | Split: 200 | Acurácia: 0.6589025285909138 | ERRO MC-NAO(N/S): 55.12698412698413 | ERRO MC-SIM(S/N): 86.87712782420303 | MC-NAO(N/S): [3473 2827] | MC-SIM (S/N): [ 424 2807] | Criterion: gini, | ALVO: HIPERTENSAO
Depht: 5 | Split: 300 | Acurácia: 0.6586926870212989 | ERRO MC-NAO(N/S): 55.142857142857146 | ERRO MC-SIM(S/N): 86.78427731352522 | MC-NAO(N/S): [3474 2826] | MC-SIM (S/N): [ 427 2804] | Criterion: gini, | ALVO: HIPERTENSAO
Depht: 5 | Split: 400 | Acurácia: 0.6586926870212989 | ERRO MC-NAO(N/S): 55.17460317460318 | ERRO MC-SIM(S/N): 86.72237697307335 | MC-NAO(N/S): [3476 2824] | MC-SIM (S/N): [ 429 2802] | Criterion: gini, | ALVO: HIPERTENSAO
Depht: 5 | Split: 500 | Acurácia: 0.6586926870212989 | ERRO MC-NAO(N/S): 55.17460317460318 | ERRO MC-SIM(S/N): 86.72237697307335 | MC-NAO(N/S): [3476 2824] | MC-SIM (S/N): [ 429 2802] | Criterion: gini, | ALVO: HIPERTENSAO
Depht: 5 | Split: 600 | Acurácia: 0.6586926870212989 | ERRO MC-NAO(N/S): 55.174603174

**2.2.1) Criação de Modelos com Árvore de Decisão (DIABETES)**

In [39]:
# Criação de modelos com ENTROPY
TP_CRITERION = 'entropy'
func_instancia_classif(TP_CRITERION, NU_DEPHT_INI, NU_DEPHT_FIM, NU_DEPHT_INC, NU_SPLIT_INI, NU_SPLIT_FIM, NU_SPLIT_INC,
                       y_train, y_test, X_train, X_test, 'DIABETES')


RESULTADOS COM ENTROPY

Depht: 5 | Split: 200 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.61904761904762 | ERRO MC-SIM(S/N): 79.10863509749304 | MC-NAO(N/S): [4008 2292] | MC-SIM (S/N): [ 675 2556] | Criterion: entropy, | ALVO: DIABETES
Depht: 5 | Split: 300 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.61904761904762 | ERRO MC-SIM(S/N): 79.10863509749304 | MC-NAO(N/S): [4008 2292] | MC-SIM (S/N): [ 675 2556] | Criterion: entropy, | ALVO: DIABETES
Depht: 5 | Split: 400 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.65079365079365 | ERRO MC-SIM(S/N): 79.04673475704116 | MC-NAO(N/S): [4010 2290] | MC-SIM (S/N): [ 677 2554] | Criterion: entropy, | ALVO: DIABETES
Depht: 5 | Split: 500 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.65079365079365 | ERRO MC-SIM(S/N): 79.04673475704116 | MC-NAO(N/S): [4010 2290] | MC-SIM (S/N): [ 677 2554] | Criterion: entropy, | ALVO: DIABETES
Depht: 5 | Split: 600 | Acurácia: 0.6887000314762355 | ERRO MC-NAO(N/S): 63.6507936

In [40]:
# Criação de modelos com GINI
TP_CRITERION = 'gini'
func_instancia_classif(TP_CRITERION, NU_DEPHT_INI, NU_DEPHT_FIM, NU_DEPHT_INC, NU_SPLIT_INI, NU_SPLIT_FIM, NU_SPLIT_INC,
                       y_train_diabetes, y_test_diabetes, X_train_diabetes, X_test_diabetes, 'DIABETES')


RESULTADOS COM GINI

Depht: 5 | Split: 200 | Acurácia: 0.6577483999580317 | ERRO MC-NAO(N/S): 64.22363166549212 | ERRO MC-SIM(S/N): 78.76106194690266 | MC-NAO(N/S): [5468 3046] | MC-SIM (S/N): [216 801] | Criterion: gini, | ALVO: DIABETES
Depht: 5 | Split: 300 | Acurácia: 0.6577483999580317 | ERRO MC-NAO(N/S): 64.22363166549212 | ERRO MC-SIM(S/N): 78.76106194690266 | MC-NAO(N/S): [5468 3046] | MC-SIM (S/N): [216 801] | Criterion: gini, | ALVO: DIABETES
Depht: 5 | Split: 400 | Acurácia: 0.6577483999580317 | ERRO MC-NAO(N/S): 64.22363166549212 | ERRO MC-SIM(S/N): 78.76106194690266 | MC-NAO(N/S): [5468 3046] | MC-SIM (S/N): [216 801] | Criterion: gini, | ALVO: DIABETES
Depht: 5 | Split: 500 | Acurácia: 0.6648830133249397 | ERRO MC-NAO(N/S): 65.12802443035001 | ERRO MC-SIM(S/N): 77.87610619469027 | MC-NAO(N/S): [5545 2969] | MC-SIM (S/N): [225 792] | Criterion: gini, | ALVO: DIABETES
Depht: 5 | Split: 600 | Acurácia: 0.6646731717553247 | ERRO MC-NAO(N/S): 65.09278834860231 | ERRO MC-SIM(S

# **3) AVALIAÇÃO DO MODELO - Escolha da melhor configuração para o Classificador**



In [0]:
# Exportando o DataFrame com os resultados para XLS
df_scores.to_excel('metricas_arvore.xls')

# 3.1) Modelo de Hipertensão - Configuração escolhida




In [42]:
df_scores[df_scores['ALVO'] == 'HIPERTENSAO'].head()

,x_depht,y_split,accuracy_score,MC - NAO(N/S),MC - SIM (S/N),Matriz Conf NAO,Matriz Conf SIM,Criterion,ALVO
0,5,200,0.6887,63.619048,79.108635,"[4008, 2292]","[675, 2556]",entropy,HIPERTENSAO
1,5,300,0.6887,63.619048,79.108635,"[4008, 2292]","[675, 2556]",entropy,HIPERTENSAO
2,5,400,0.6887,63.650794,79.046735,"[4010, 2290]","[677, 2554]",entropy,HIPERTENSAO
3,5,500,0.6887,63.650794,79.046735,"[4010, 2290]","[677, 2554]",entropy,HIPERTENSAO
4,5,600,0.6887,63.650794,79.046735,"[4010, 2290]","[677, 2554]",entropy,HIPERTENSAO


In [43]:
KF = KFold(n_splits=10, random_state=7)
clf_dtree_hiper = DecisionTreeClassifier(criterion = 'gini', max_depth = 30, min_samples_split = 1200)
clf_dtree_hiper = clf_dtree_hiper.fit(X_train, y_train)

#Acurácia com Cross Validation e K-Folds
resultado = cross_val_score(clf_dtree_hiper, X_train, y_train, cv=KF, scoring='accuracy')
acc_dtree_hiper = resultado.mean()
#df_comp.loc[len(df_comp)] = ['Decisao Acuracia - CV', resultado.mean() , '']
print(acc_dtree_hiper)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.6696674944527599


**Geração da Árvore Gráfica**

In [0]:
import pydotplus
import graphviz

In [0]:
arvore_grafica = export_graphviz( 
         clf_dtree_hiper, 
         out_file=None,
         feature_names=X_test.columns,
         class_names=['(0) NÃO HIPERTENSO','(1) HIPERTENSO'],  
         filled=True, 
         rounded=True,
         proportion=True,
         node_ids=True,
         rotate=False,
         label='all',
         special_characters=True
        )  

In [46]:
graph = pydotplus.graph_from_dot_data(arvore_grafica)  
graph.write_png('arvore_grafica_hipertenso.png')

True

# 3.1) Modelo de Diabetes - Configuração escolhida

In [47]:
df_scores[df_scores['ALVO'] == 'DIABETES'].head()

,x_depht,y_split,accuracy_score,MC - NAO(N/S),MC - SIM (S/N),Matriz Conf NAO,Matriz Conf SIM,Criterion,ALVO
380,5,200,0.6887,63.619048,79.108635,"[4008, 2292]","[675, 2556]",entropy,DIABETES
381,5,300,0.6887,63.619048,79.108635,"[4008, 2292]","[675, 2556]",entropy,DIABETES
382,5,400,0.6887,63.650794,79.046735,"[4010, 2290]","[677, 2554]",entropy,DIABETES
383,5,500,0.6887,63.650794,79.046735,"[4010, 2290]","[677, 2554]",entropy,DIABETES
384,5,600,0.6887,63.650794,79.046735,"[4010, 2290]","[677, 2554]",entropy,DIABETES


In [48]:
KF = KFold(n_splits=10, random_state=7)
clf_dtree_diab = DecisionTreeClassifier(criterion = 'entropy', max_depth = 15, min_samples_split = 1100)
clf_dtree_diab = clf_dtree_diab.fit(X_train_diabetes, y_train_diabetes)
resultado = cross_val_score(clf_dtree_diab, X_train_diabetes, y_train_diabetes, cv=KF, scoring='accuracy')

acc_dtree_diab = resultado.mean()
#df_comp.loc[len(df_comp)] = ['Decisao Acuracia - CV', resultado.mean() , '']
print(acc_dtree_diab)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.6867462034807195


**Geração da Árvore Gráfica**

In [0]:
import pydotplus
import graphviz

In [0]:
arvore_grafica = export_graphviz( 
         clf_dtree_diab, 
         out_file=None,
         feature_names=X_test.columns,
         class_names=['(0) NÃO DIABÉTICO','(1) DIABÉTICO'],  
         filled=True, 
         rounded=True,
         proportion=True,
         node_ids=True,
         rotate=False,
         label='all',
         special_characters=True
        )  

In [51]:
graph = pydotplus.graph_from_dot_data(arvore_grafica)  
graph.write_png('arvore_grafica_diabetes.png')

True

# **4) CRIAÇÃO DO MODELO DE REDE NEURAL ARTIFICIAL - HIPERTENSÃO**

In [0]:
#from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [0]:
# Criação do DataFrame Vazio para armazenamento de todas as configurações e resultados dos modelos de 
#árvore de decisão implementados
df_scores_rna = pd.DataFrame(columns=['NU_ITERACAO', 'ACURACIA', 'ACERTO - MC NAO', 'ACERTO - MC SIM', 'MATRIZ CONFUSAO', 'IN_ALVO'])

In [0]:
def gera_classificador_RNA(MAX_INTER, MAX_INTER_FIM, MAX_INTER_INT, Y_TRAIN, Y_TEST, X_TRAIN, X_TEST, IN_ALVO):                           
    print('Inicio da função')
    for X_MAX_INTER in range(MAX_INTER, MAX_INTER_FIM, MAX_INTER_INT):
        print('Execucao {} de {} '.format(X_MAX_INTER, MAX_INTER_FIM-1))
        clf = MLPClassifier(hidden_layer_sizes=100, activation='relu', solver='adam', alpha=0.0001, batch_size='auto', verbose='False',
                            max_iter=X_MAX_INTER)
        clf = clf.fit(X_TRAIN, Y_TRAIN)
        resultado = clf.predict(X_TEST)
        acuracia = accuracy_score(Y_TEST, resultado)
        mat_conf = confusion_matrix(Y_TEST, resultado)
        mat_conf_nao = mat_conf[0][0] / (mat_conf[0][0] + mat_conf[0][1]) * 100
        mat_conf_sim = mat_conf[1][1] / (mat_conf[1][0] + mat_conf[1][1]) * 100
        df_scores_rna.loc[len(df_scores_rna)] = [X_MAX_INTER, acuracia, mat_conf_nao, mat_conf_sim, mat_conf, IN_ALVO]
    print('Fim da função')

# Classificação Rede Neural - Hipertensão

In [56]:
gera_classificador_RNA(1, 30, 1, y_train, y_test, X_train, X_test, 'HIPERTENSAO')

Inicio da função
Execucao 1 de 29 
Iteration 1, loss = 0.56891636
Execucao 2 de 29 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.57068053
Iteration 2, loss = 0.53706410
Execucao 3 de 29 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.57722840
Iteration 2, loss = 0.53669449
Iteration 3, loss = 0.53412616
Execucao 4 de 29 


...

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (27) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.57364150
Iteration 2, loss = 0.53661552
Iteration 3, loss = 0.53498642
Iteration 4, loss = 0.53303932
Iteration 5, loss = 0.53209818
Iteration 6, loss = 0.53095737
Iteration 7, loss = 0.52974231
Iteration 8, loss = 0.52805448
Iteration 9, loss = 0.52628096
Iteration 10, loss = 0.52494220
Iteration 11, loss = 0.52229121
Iteration 12, loss = 0.52125119
Iteration 13, loss = 0.51893692
Iteration 14, loss = 0.51661911
Iteration 15, loss = 0.51298445
Iteration 16, loss = 0.51076561
Iteration 17, loss = 0.50979351
Iteration 18, loss = 0.50509596
Iteration 19, loss = 0.50343033
Iteration 20, loss = 0.50014700
Iteration 21, loss = 0.49843648
Iteration 22, loss = 0.49744503
Iteration 23, loss = 0.49354777
Iteration 24, loss = 0.49304846
Iteration 25, loss = 0.48990458
Iteration 26, loss = 0.48800151
Iteration 27, loss = 0.48570783
Iteration 28, loss = 0.48336615
Execucao 29 de 29 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (28) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.57451954
Iteration 2, loss = 0.53730402
Iteration 3, loss = 0.53435869
Iteration 4, loss = 0.53282548
Iteration 5, loss = 0.53112759
Iteration 6, loss = 0.52995781
Iteration 7, loss = 0.52779068
Iteration 8, loss = 0.52603406
Iteration 9, loss = 0.52324135
Iteration 10, loss = 0.52221026
Iteration 11, loss = 0.51909950
Iteration 12, loss = 0.51667309
Iteration 13, loss = 0.51464839
Iteration 14, loss = 0.51068879
Iteration 15, loss = 0.50889884
Iteration 16, loss = 0.50650193
Iteration 17, loss = 0.50497239
Iteration 18, loss = 0.50208340
Iteration 19, loss = 0.50062517
Iteration 20, loss = 0.49712071
Iteration 21, loss = 0.49477869
Iteration 22, loss = 0.49483894
Iteration 23, loss = 0.49068309
Iteration 24, loss = 0.48783680
Iteration 25, loss = 0.48703716
Iteration 26, loss = 0.48382712
Iteration 27, loss = 0.48246410
Iteration 28, loss = 0.47904384
Iteration 29, loss = 0.47913775
Fim da função


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (29) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [57]:
df_scores_rna.head()

,NU_ITERACAO,ACURACIA,ACERTO - MC NAO,ACERTO - MC SIM,MATRIZ CONFUSAO,IN_ALVO
0,1,0.724793,73.873016,69.761684,"[[4654, 1646], [977, 2254]]",HIPERTENSAO
1,2,0.725212,72.809524,71.959146,"[[4587, 1713], [906, 2325]]",HIPERTENSAO
2,3,0.707586,66.492063,79.077685,"[[4189, 2111], [676, 2555]]",HIPERTENSAO
3,4,0.705802,66.015873,79.480037,"[[4159, 2141], [663, 2568]]",HIPERTENSAO
4,5,0.720386,71.031746,74.001857,"[[4475, 1825], [840, 2391]]",HIPERTENSAO


In [58]:
KF = KFold(n_splits=10, random_state=7)
clf_rna_hiper = MLPClassifier(hidden_layer_sizes=100, activation='relu', solver='adam', alpha=0.0001, batch_size='auto', max_iter=10)
resultado = cross_val_score(clf_rna_hiper, X_train, y_train, cv=KF, scoring='accuracy')

acc_rna_hip = resultado.mean()
#df_comp.loc[len(df_comp)] = ['Decisao Acuracia - CV', resultado.mean() , '']
print(acc_rna_hip)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_i

0.6578908750618156


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Classificação Rede Neural - Diabetes

In [59]:
gera_classificador_RNA(1, 30, 1, y_train_diabetes, y_test_diabetes, X_train_diabetes, X_test_diabetes, 'DIABETES')

Inicio da função
Execucao 1 de 29 
Iteration 1, loss = 0.55291382
Execucao 2 de 29 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.55040234
Iteration 2, loss = 0.51221649
Execucao 3 de 29 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.54648096
Iteration 2, loss = 0.51244643
Iteration 3, loss = 0.51017656
Execucao 4 de 29 


...

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (27) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.55283188
Iteration 2, loss = 0.51353278
Iteration 3, loss = 0.50796086
Iteration 4, loss = 0.50449088
Iteration 5, loss = 0.50010343
Iteration 6, loss = 0.49374814
Iteration 7, loss = 0.48730616
Iteration 8, loss = 0.47869440
Iteration 9, loss = 0.46885255
Iteration 10, loss = 0.46060848
Iteration 11, loss = 0.45028500
Iteration 12, loss = 0.44284381
Iteration 13, loss = 0.43584747
Iteration 14, loss = 0.42604446
Iteration 15, loss = 0.41705381
Iteration 16, loss = 0.41091760
Iteration 17, loss = 0.40375676
Iteration 18, loss = 0.39736266
Iteration 19, loss = 0.39201554
Iteration 20, loss = 0.38437441
Iteration 21, loss = 0.37854018
Iteration 22, loss = 0.37292307
Iteration 23, loss = 0.36929990
Iteration 24, loss = 0.36282947
Iteration 25, loss = 0.35925615
Iteration 26, loss = 0.35324746
Iteration 27, loss = 0.34957068
Iteration 28, loss = 0.34477711
Execucao 29 de 29 


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (28) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.55284449
Iteration 2, loss = 0.51317379
Iteration 3, loss = 0.50664748
Iteration 4, loss = 0.50129292
Iteration 5, loss = 0.49372215
Iteration 6, loss = 0.48529808
Iteration 7, loss = 0.47672291
Iteration 8, loss = 0.46775687
Iteration 9, loss = 0.45847948
Iteration 10, loss = 0.44875475
Iteration 11, loss = 0.43941491
Iteration 12, loss = 0.43167761
Iteration 13, loss = 0.42223703
Iteration 14, loss = 0.41468465
Iteration 15, loss = 0.40613409
Iteration 16, loss = 0.39707634
Iteration 17, loss = 0.38995077
Iteration 18, loss = 0.38340456
Iteration 19, loss = 0.37660172
Iteration 20, loss = 0.36960553
Iteration 21, loss = 0.36574674
Iteration 22, loss = 0.35736955
Iteration 23, loss = 0.35476929
Iteration 24, loss = 0.34969100
Iteration 25, loss = 0.34250630
Iteration 26, loss = 0.33793145
Iteration 27, loss = 0.33307192
Iteration 28, loss = 0.32924023
Iteration 29, loss = 0.32326140
Fim da função


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (29) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [60]:
df_scores_rna.head()

,NU_ITERACAO,ACURACIA,ACERTO - MC NAO,ACERTO - MC SIM,MATRIZ CONFUSAO,IN_ALVO
0,1,0.724793,73.873016,69.761684,"[[4654, 1646], [977, 2254]]",HIPERTENSAO
1,2,0.725212,72.809524,71.959146,"[[4587, 1713], [906, 2325]]",HIPERTENSAO
2,3,0.707586,66.492063,79.077685,"[[4189, 2111], [676, 2555]]",HIPERTENSAO
3,4,0.705802,66.015873,79.480037,"[[4159, 2141], [663, 2568]]",HIPERTENSAO
4,5,0.720386,71.031746,74.001857,"[[4475, 1825], [840, 2391]]",HIPERTENSAO


In [61]:
KF = KFold(n_splits=10, random_state=7)
clf_rna_diab = MLPClassifier(hidden_layer_sizes=100, activation='relu', solver='adam', alpha=0.0001, batch_size='auto', max_iter=10)
resultado = cross_val_score(clf_rna_diab, X_train_diabetes, y_train_diabetes, cv=KF, scoring='accuracy')

acc_rna_diab = resultado.mean()
#df_comp.loc[len(df_comp)] = ['Decisao Acuracia - CV', resultado.mean() , '']
print(acc_rna_diab)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_i

0.7405714806225677


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
# Exportando o DataFrame com os resultados para XLS
df_scores_rna.to_excel('metricas_rna.xls')

In [0]:
df_comp.loc[len(df_comp)] = ['CV - Acurácia DTree', acc_dtree_hiper , acc_dtree_diab]
df_comp.loc[len(df_comp)] = ['CV - Acurácia RNA (MLP)', acc_rna_hip , acc_rna_diab]


In [64]:
print(df_comp)


       *******************  HIPERTENSÃO  DIABETES
0      CV - Acurácia DTree     0.669667  0.686746
1  CV - Acurácia RNA (MLP)     0.657891  0.740571
